# Worksheet 08

Name:  Dima Kazlouski
UID: U49144585

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[6.699352831676877, 4.799666698952097, 4.401761897722514, 4.507250753289579, 3.12781498039314, 3.776428853518498, 3.694520271239159, 4.961042569289604, 2.1896480385305463, 5.2727316172409955]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
c2 = np.random.normal(8, 1, 10).tolist()
print(c2)

[6.267357821473129, 7.809707070081489, 9.050995924518555, 9.199430850476809, 8.08751383593244, 9.058814319669226, 8.870876172839125, 7.670286315496307, 8.750709320357622, 6.482236136212417]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [4]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[5.2727316172409955, 6.482236136212417, 2.1896480385305463, 4.961042569289604, 8.750709320357622, 3.694520271239159, 3.776428853518498, 3.12781498039314, 7.670286315496307, 4.507250753289579]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

c1 has a mean of 5 and stdev 1. c2 has a mean of 8 and a stdev of 1

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [5]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , 
        sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))


P(C_1) = 0.7,  P(C_2) = 0.3
mean_1 = 3.9327767262145032,  mean_2 = 7.634410590688783
var_1 = 0.9929288559287395,  var_2 = 0.8583052983462136


c:\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


The means estimated are very close while the variance is a bit off

e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [17]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x)
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x)


probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  5.2727316172409955
probability of observing that point if it came from cluster 0 =  0.16163511489470617
probability of observing that point if it came from cluster 1 =  0.010549405567834695
point =  6.482236136212417
probability of observing that point if it came from cluster 0 =  0.01487361172524841
probability of observing that point if it came from cluster 1 =  0.18878637467032333
point =  2.1896480385305463
probability of observing that point if it came from cluster 0 =  0.08605171764860012
probability of observing that point if it came from cluster 1 =  8.490528383412294e-10
point =  4.961042569289604
probability of observing that point if it came from cluster 0 =  0.23502481250434681
probability of observing that point if it came from cluster 1 =  0.003636122099919707
point =  8.750709320357622
probability of observing that point if it came from cluster 0 =  3.1004112438456495e-06
probability of observing that point if it came from cluster 1 =  0.19950751327019392
point 

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [18]:
prob_c = [sum(prob_c0_x) / len(prob_c0_x), sum(prob_c1_x) / len(prob_c1_x)]

mean = []
mean.append(sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x))
mean.append(sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x))

var = []
var.append(sum([prob_c0_x[i] * (data[i] - mean[0])**2 for i in range(len(data))]) / sum(prob_c0_x))
var.append(sum([prob_c1_x[i] * (data[i] - mean[1])**2 for i in range(len(data))]) / sum(prob_c1_x))


print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.7122411690601914,  P(C_2) = 0.28775883093980864
mean_1 = 3.9831410597373544,  mean_2 = 7.667218529600239
var_1 = 1.110595235178727,  var_2 = 0.893980635090848


Very close to the original values.

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [20]:
# Compute updated P(C_j | X_i) using the new parameter estimates

prob_c0_x_new = [] 
prob_c1_x_new = []  

for p in data:
    pdf_i = []

    for j in range(k):
        pdf_i.append(norm.pdf(p, mean[j], np.sqrt(var[j])))

    prob_x_i = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    prob_c0_x_new.append(pdf_i[0] * prob_c[0] / prob_x_i)
    prob_c1_x_new.append(pdf_i[1] * prob_c[1] / prob_x_i)

probs_new = list(zip(data, prob_c0_x_new, prob_c1_x_new))
probs_new


[(5.2727316172409955, 0.9628829561888023, 0.037117043811197736),
 (6.482236136212417, 0.22642564955364328, 0.7735743504463567),
 (2.1896480385305463, 0.9999999012486159, 9.875138414279613e-08),
 (4.961042569289604, 0.9886061544852868, 0.011393845514713135),
 (8.750709320357622, 0.00015395102962183205, 0.9998460489703781),
 (3.694520271239159, 0.9999314106987797, 6.858930122033018e-05),
 (3.776428853518498, 0.9999034483380013, 9.655166199876024e-05),
 (3.12781498039314, 0.99999381517979, 6.18482021001454e-06),
 (7.670286315496307, 0.004855410801576688, 0.9951445891984233),
 (4.507250753289579, 0.9980903739417072, 0.0019096260582927114)]

Very similar from previous estimate suggesting that parameters converged quickly. 

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [24]:
cluster_assignments = [0 if prob_c0 > prob_c1 else 1 for prob_c0, prob_c1 in zip(prob_c0_x_new, prob_c1_x_new)]

clustered_data = list(zip(data, cluster_assignments))
clustered_data


[(5.2727316172409955, 0),
 (6.482236136212417, 1),
 (2.1896480385305463, 0),
 (4.961042569289604, 0),
 (8.750709320357622, 1),
 (3.694520271239159, 0),
 (3.776428853518498, 0),
 (3.12781498039314, 0),
 (7.670286315496307, 1),
 (4.507250753289579, 0)]

### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      Disagree       |
| A  C |      Agree       |
| A  D |      Agree       |
| A  E |      Agree       |
| B  C |      Disagree       |
| B  D |      Agree       |
| B  E |      Agree       |
| C  D |      Agree       |
| C  E |      Agree       |
| D  E |      Disagree       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

$${N\choose2} = {{N!}\over{2!*(N-2)!}}$$

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

36


d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

11 agreements, 25 disagreements

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.